# 走行

このNotebookで自動走行をおこないます。

## Jetsonの認識

In [ ]:
import Jetson.GPIO as GPIO

BOARD_NAME=GPIO.gpio_pin_data.get_data()[0]
if BOARD_NAME == "JETSON_NX":
    print("Jetson Xavier NXを認識")
    I2C_BUSNUM = 8
    MODE = 2
elif BOARD_NAME == "JETSON_XAVIER":
    print("Jetson AGX Xavierを認識")
    I2C_BUSNUM = 8
    MODE = 2
elif BOARD_NAME == "JETSON_NANO":
    print("Jetson Nanoを認識")
    I2C_BUSNUM = 1
    MODE = 0
elif BOARD_NAME == "JETSON_ORIN":
    print("Jetson AGX Orinを認識")
    I2C_BUSNUM = 7
    MODE = 0
elif BOARD_NAME == "JETSON_ORIN_NANO":
    print("Jetson Orin Nanoを認識")
    I2C_BUSNUM = 7
    MODE = 0

In [ ]:
!echo "jetson" | sudo -S nvpmodel -m $MODE

In [ ]:
!echo "jetson" | sudo -S nvpmodel -q

In [ ]:
!echo "jetson" | sudo -S jetson_clocks

## ログの表示用 Widget

In [ ]:
import ipywidgets
from ipywidgets import Button, Layout, Textarea, HBox, VBox, Label
import os
import glob
from IPython.display import clear_output

l = Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
process_widget = ipywidgets.Textarea(description='ログ', value='', layout=l)

process_no = 0
def write_log(msg):
    global process_widget, process_no
    process_no = process_no + 1
    process_widget.value = str(process_no) + ": " + msg + "\n" + process_widget.value
    
    # UIのクリアと更新
    clear_output(wait=True)

## PWMの値の読み込み

In [ ]:
import Fabo_PCA9685
import time
import pkg_resources
import smbus
import time
import json

SMBUS='smbus'
BUSNUM=I2C_BUSNUM
SERVO_HZ=60
INITIAL_VALUE=300
bus = smbus.SMBus(BUSNUM)
PCA9685 = Fabo_PCA9685.PCA9685(bus,INITIAL_VALUE,address=0x40)
PCA9685.set_hz(SERVO_HZ)

STEERING_CH = 0
THROTTLE_CH = 1
direction = 0
REVERSE = 0
NORMAL = 1

pwm_front = 0
pwm_back = 0

with open('pwm_params.json') as f:
    json_str = json.load(f)
    
    pwm_stop = json_str["pwm_speed"]["stop"]
    pwm_front = json_str["pwm_speed"]["front"]
    pwm_back = json_str["pwm_speed"]["back"]
    pwm_left = json_str["pwm_steering"]["left"]
    pwm_center = json_str["pwm_steering"]["center"]
    pwm_right = json_str["pwm_steering"]["right"]

    
if pwm_front >= pwm_back:
    direction = REVERSE
else:
    direction = NORMAL
    
PCA9685.set_channel_value(STEERING_CH, pwm_center)
PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)

## カメラの読込

この部分でエラーが発生する場合は、Jetsonの再起動をお願いします。<br>
それでも、カメラが認識できない場合は、ケーブルの接続確認をしてください。


In [ ]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=30)

## 走行処理

In [ ]:
import torch
import threading
from utils import preprocess
import subprocess
import cv2
import time
from torch2trt import TRTModule
import subprocess
import datetime

IMG_WIDTH=224

model_widget = ipywidgets.Dropdown(options=[],description='モデル')
model_time_widget = ipywidgets.Label(description='作成日時')
load_button = ipywidgets.Button(description='Load')
run_button = ipywidgets.Button(description='Run')
stop_button = ipywidgets.Button(description='Stop')
pwm_left_widget = ipywidgets.IntText(value=pwm_left,description='PWM 左')
pwm_center_widget = ipywidgets.IntText(value=pwm_center,description='PWM 中央')
pwm_right_widget = ipywidgets.IntText(value=pwm_right,description='PWM 右')
check_left_button = ipywidgets.Button(description='チェック左')
check_center_button = ipywidgets.Button(description='チェック中央')
check_right_button = ipywidgets.Button(description='チェック右')
pwm_stop_widget = ipywidgets.IntText(value=pwm_stop,description='PWM 停止')
name_widget = ipywidgets.Text(description='映像の保存名')
record_box = ipywidgets.Checkbox(False, description='録画')
speed_gain_slider = ipywidgets.FloatSlider(description='Speed gain', min=0.1, max=2.0, step=0.1, value=1.0, orientation='horizontal')
speed_raw_slider = ipywidgets.IntSlider(description='Speed raw', min=1, max=224, step=1, value=80, orientation='horizontal')
steering_gain_slider = ipywidgets.FloatSlider(description='Steering gain', min=0.1, max=2.0, step=0.1, value=1.0, orientation='horizontal')
speed_value_box = ipywidgets.Checkbox(True, description='Speedに固定値')
speed_ai_box = ipywidgets.Checkbox(False, description='Speedに推論値')

record = False
running = False
load = False

def map_rc(x, in_min, in_max, out_min, out_max):
    return (x - in_min) * (out_max - out_min) // (in_max - in_min) + out_min

def handle(x):
    global pwm_right,pwm_left,STEERING_CH,PCA9685
    x = map_rc(x, 224, 0, pwm_right, pwm_left)
    PCA9685.set_channel_value(STEERING_CH, x)
    
def normal_throttle(speed):
    global pwm_front,pwm_back,THROTTLE_CH,PCA9685
    speed = map_rc(speed, 224, 0, pwm_front, pwm_stop)
    PCA9685.set_channel_value(THROTTLE_CH, speed)
    
def reverse_throttle(speed):
    global pwm_front,pwm_back,THROTTLE_CH,PCA9685
    speed = map_rc(speed, 224, 0, pwm_back, pwm_stop)
    PCA9685.set_channel_value(THROTTLE_CH, speed)

def live():
    global speed_ai_flag,running,camera,IMG_WIDTH,record,model_trt,preprocess,pwm_stop,save_dir,num,cv2,count,speed_gain_slider,steering_gain_slider
    
    count = 1
    num = 1
    frame_count = 0

    start_time = time.time()
   
    while running:
        image = camera.read()
        if record == True:
            remarked_img = image.copy()
        image = preprocess(image).half()
        output = model_trt(image).detach().cpu().numpy().flatten()
        x = float(output[0]) * steering_gain_slider.value
        y = float(output[1])
        x = int(IMG_WIDTH * (x / 2.0 + 0.5))
        y = int(IMG_WIDTH * (y / 2.0 + 0.5))
        handle(x)

        speed = float(output[3])
        speed = int(IMG_WIDTH * (speed / 2.0 + 0.5)) * speed_gain_slider.value
        
        if speed_ai_flag == False:
            speed = speed_raw_slider.value
        else:
            if speed > 224:
                speed = 224
            
        # write_log("Speed:" + str(speed))
        
        if direction == REVERSE:
            reverse_throttle(speed)
        else:
            normal_throttle(speed)
            
        if record == True:
            if num % 2 == 0:
                name = "0_0_{:0=5}.jpg".format(num)
                image_path = os.path.join(save_dir, name)
                cv2.imwrite(image_path, remarked_img)
            num += 1
        count += 1
        frame_count += 1

        if time.time() - start_time > 3.0:
            fps = frame_count / 3.0
            speed_type = ""
            if speed_ai_flag == False:
                speed_type = f"(固定)"
            else:
                speed_type = f"(推論)"
            write_log(f"Speed: {speed} {speed_type}, Steering Gain: {steering_gain_slider.value}, FPS: {fps:.1f} (3秒ごとに表示)")
            frame_count = 0
            start_time = time.time()
        
def run(change):
    global running,execute_thread,name_widget,save_dir,start_time,load,speed_gain_slider,steering_gain_slider
    
    if load == False:
        write_log("モデルが読み込まれていません")
        return

    if running == False:
        if record == True:
            if name_widget.value != "":
                save_dir = "run/" + name_widget.value + "/xy/" 
                if not os.path.exists(save_dir):
                    subprocess.call(['mkdir', '-p', save_dir])
                write_log(save_dir + "にデータを保存します。")            

                running = True
                execute_thread = threading.Thread(target=live)
                execute_thread.start()
                start_time = time.time()
                write_log("AIが起動しました。")
            else:
                write_log("【Error】 映像の保存先を入力してください。")
        else:
            running = True
            execute_thread = threading.Thread(target=live)
            execute_thread.start()
            start_time = time.time()
            write_log("AIが起動しました。") 

def stop(change):
    global running,execute_thread,end_time,start_time,count,pwm_stop
    if running == True:
        PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
        try:
            end_time = time.time() - start_time
            fps = count/int(end_time)
            process_time = int((end_time/count)*1000)
        except:
            fps = -1
            process_time = -1
        
        write_log("AIを停止しました。")
        write_log("処理結果:FPS: " + str(round(fps,2)) + ",処理回数: " + str(count) + ",　処理時間(1回平均値): " + str(process_time) + " ms")
        running = False
        execute_thread.join()
    else:
        PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
        write_log("現在AIは動いていません。")

def load_model(change):
    global model_trt,model_widget,load
    try:
        write_log(model_widget.value + "の読込を実行します(初回は時間がかかります)。")
        model_trt = TRTModule()
        model_trt.load_state_dict(torch.load(model_widget.value))
        write_log(model_widget.value + "の読込に成功しました。")
        load = True
    except:
        write_log("【Error】" + model_widget.value + "の読込に失敗しました。")
        
def model_list(change):
    global model_widget
    try:
        files = glob.glob('./model_trt/*.pth', recursive=True)
        model_widget.options = files
        ts = os.path.getctime(files[0])
        d = datetime.datetime.fromtimestamp(ts)
        s = d.strftime('%Y-%m-%d %H:%M:%S')
        model_time_widget.value = s
    except:
        model_widget.options = []
model_list("list")

def change_file(change):
    file = model_widget.value
    ts = os.path.getctime(file)
    d = datetime.datetime.fromtimestamp(ts)
    s = d.strftime('%Y-%m-%d %H:%M:%S')
    model_time_widget.value = s
model_widget.observe(change_file, names='value')

def on_video(change):
    global record
    record^=True

SpeedInference = False

def on_speed_ai(c):
    global speed_ai_flag,speed_ai_box,speed_value_box
    speed_value_box.value = False
    speed_ai_flag = True
    
def on_speed_value(c):
    global speed_ai_flag,speed_ai_box,speed_value_box
    speed_ai_box.value = False
    speed_ai_flag = False

speed_ai_box.observe(on_speed_ai)
speed_value_box.observe(on_speed_value)

load_button.on_click(load_model)
run_button.on_click(run)
stop_button.on_click(stop)
record_box.observe(on_video)

走行までの流れは以下の通りです。

1. <b>学習済みモデルを指定してLoadする</b><br>
2. <b>Speedに固定値, Speedに推論値のいずれかにチェックマークをいれる</b><br>
固定値を選んだ場合は固定の速度で走り続けます。推論値を選んだ推論結果を速度に反映します。速度用のアノテーションは14_remakrk.ipynbで追加可能です。<br>
3. <b>[オプション] 走行動画を録画する場合は、録画にチェックマークをいれて、保存ファイル名を指定する</b><br>
./runフォルダに保存<br>
4. <b>runボタンを押して、プロポの裏側のボタンを押して AIモードで自動走行開始する</b><br>
5. <b>終了時は、stopボタンを押す</b><br>
録画のチェックマークがついている場合は、stopで録画も終了<br>
Speed Inferenceのチェックマークがついている場合は、スロットル量の推論も有効になる<br>
<br>
カメラが60fpsで動いている場合は16ms以内、カメラが30fpsで動いている場合は、33ms以内での処理完了が正常な挙動となります。

In [ ]:
data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([model_widget,model_time_widget,load_button]),
    ipywidgets.HBox([steering_gain_slider]),
    ipywidgets.HBox([speed_value_box,speed_raw_slider]),
    ipywidgets.HBox([speed_ai_box,speed_gain_slider]),
    ipywidgets.HBox([record_box,name_widget]),
    ipywidgets.HBox([run_button, stop_button]),
    process_widget
])
display(data_collection_widget)

### カメラの終了処理(必須)

In [ ]:
import time

camera.running = False
camera.cap.release()